In [3]:
import json

In [4]:
from openai import OpenAI

from agents.adjudicator import query as consult_players_handbook

In [5]:
import re
import random

from typing import List


def dice_roll(query: str) -> List[int]:
    results = re.search(r"^(\d+)d(\d+)(\s?(\+|-)\s?\d+)?$", query)
    if not results:
        raise ValueError(f"query {query} is uninterpretable")

    num = results.group(1)
    val = results.group(2)
    modifier = results.group(3)

    # TODO: return signature here is inconsistent. The issue is we haven't nailed down how the caller is asking for rolls.
    #  For example, when rolling with advantage, it sometimes calls for 2d20 and keeps the higher, and sometimes it calls for 2d20kh1, i.e. expecting us to do that here
    #
    # In other cases, the intent is not to discard a roll but to sum them, e.g. when rolling for damage it might call for 3d6+3
    result = [random.randint(1, int(val)) for i in range(int(num))]
    if modifier:
        modifier = int(modifier.strip()[1:]) * (-1 if modifier.strip()[0] == "-" else 1)
        return sum(result) + modifier
    else:
        return result

In [6]:
pc_tools = [
  {
      "type": "function",
      "function": {
          "name": "consult_players_handbook",
          "description": "Ask a question pertaining to the rules, setting, lore, or current situtation. Call this whenever you need more information.",
          "parameters": {
              "type": "object",
              "properties": {
                  "question": {
                      "type": "string",
                      "description": "A question that should be answered by consulting the available reference materials. Share all relevant details.",
                  },
              },
              "required": ["question"],
              "additionalProperties": False,
          },
      }
  }  
]

npc_tools = [
  {
      "type": "function",
      "function": {
          "name": "consult_players_handbook",
          "description": "Ask a question pertaining to the rules, setting, lore, or current situtation. Call this whenever you need more information.",
          "parameters": {
              "type": "object",
              "properties": {
                  "question": {
                      "type": "string",
                      "description": "A question that should be answered by consulting the available reference materials. Share all relevant details.",
                  },
              },
              "required": ["question"],
              "additionalProperties": False,
          },
      }
  },
  {
      "type": "function",
        "function": {
            "name": "dice_roller",
            "description": "Roll a dice with a given number of sides.",
            "parameters": {
                "type": "object",
                "properties": {
                    "dice": {
                        "type": "string",
                        "description": "The total number of dice and the number of sides on each die, e.g. '2d6'.",
                    },
                },
                "required": ["dice"],
                "additionalProperties": False,
            },
        }
  }
]

In [41]:
pc_info = """
Cedric van Zorndt (the Player)
Human Rogue Lvl 1
Armor Class 14
Hit Points 9
STR 12 (+1) - DEX 16 (+3) - CON 13 (+1) - INT 9 (-1) - WIS 12 (+1) - CHA 15 (+2)
Proficiency bonus: +2
Proficiencies:
- Armor: light armor
- Weapons: Simple weapons, hand crossbows, longswords, rapiers, shortswords
- Tools: Thieves' tools
Saving Throws: Dexterity, Intelligence
Skills: Acrobatics, Deception, Sleight of Hand, Stealth
Equipment: Shortsword, shortbow with a quiver of 20 arrows, burglar's pack, leather armor, two daggers, thieves' tools
Expertise: Stealth, Thieves' tools
Sneak attack 1d6
    """

npc_info = """
Imp (NPC)
Armor Class 13
Hit Points 10
STR 6 (-2) - DEX 17 (+3) - CON 13 (+1) - INT 11 (+0) - WIS 12 (+1) - CHA 14 (+2)
Speed 20 (Fly 40) (20 in Rat form, 20 fly 60 in Raven form, 20 climb 20 in Spider form)
Skills: Deception +4 Insight +3 Persusasion +4 Stealth +5
Damage Resistances: Cold; bludgeoning, piercing and slashing from non-magical attacks that aren't silvered
Damage Immunities: Fire, poison
Condition Immunities: poisoned
Senses: darkvision 120ft; Passive perception 11
Languages: Infernal, Common
Challenge: 1 (200 XP)
Shapechanger: Te imp can use its action to polymorph into the beast form of a rat, a raven, or a spider, or into its devil form. Its statistics are
the same for each form, except for the speed changes noted. Any equipment it is wearing or carrying is not transformed. It reverts to its devil form if it dies.
Devil's Sight: Magical darkness doesn't impede the imp's darkvision.
Magic Resistance: The imp has advantage on saving tghrows against spells and other magical effects.
Actions:
- Sting (Bite in Beast Form). Melee weapon Attack: +5 to hit, reach 5 ft, one creature. Hit: 5 (1d4+3) piercing damage, and the target must make a DC 11 Constitution saving throw, taking 10 (3d6) poison damage on a failed save, or half as much on a successful one.
- Invisibility. The imp magically turns invisible until it attacks or until its concentration ends (as if concentrating on a spell). Any equipment the imp wears or carries is invisible with it.
"""

In [44]:
player_message_content = """
      You are a Dungeon Master overseeing a combat encounter. Do not assume player actions.  The player will tell you their action,
      and you will narrate the outcome.

      IMPORTANT: Use only the information you have been given to narrate the outcome. Whenever a question cannot be resolved with the information
       you have been given, you MUST use the consult_players_handbook tool to get up-to-date information. This includes any information about the rules of the game,
       the setting, the mechanics, NPC stat sheets, and so on.
"""

npc_message_content= """
    You are a Dungeon Master overseeing a combat encounter. Do not assume player actions.
    Use only the information you have been given to make decisions and narrate the outcome. Whenever a question cannot be resolved with the information
       you have been given, you MUST the consult_players_handbook tool to get up-to-date information. This includes any information about the rules of the game,
       the setting, the mechanics, NPC stat sheets, and so on.

    You will act as the NPC in the encounter. You will narrate its action and determine the outcomes of that action.
    The NPC gets only ONE action. If it calls for a roll, use the dice_roller tool.
"""  

In [45]:
initiating_action =   {
      "role": "user",
      "content": "I attack the imp with my shortsword"
  }

In [46]:
client = OpenAI()

def call_open_ai(messages, tools):
  print(f"Content of message: {messages[-1]}")
  return client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
  )

In [47]:
def resolve_attack(developer_message, tools, conversations = None):
    if not conversations:
        conversations = []
    total_tool_calls = 0 
    while True:
        # the basic idea is, we are keeping track of conversations which follow the initial developer instruction
        #  and we will pass in different sets of developer instructions depending on the context. But the conversations
        #  array will be the same throughout the encounter.
        response = call_open_ai(developer_message + conversations, tools)
        print(f"Response is: {response}")
        conversations.append(response.choices[0].message)
        # If the response has a tool call, call the tool
        if response.choices[0].message.tool_calls:
            for tc in response.choices[0].message.tool_calls:
                if tc.function.name == "consult_players_handbook":
                    tool_result = consult_players_handbook(json.loads(tc.function.arguments)["question"])
                    conversations.append({
                        "role": "tool",
                        "content": tool_result,
                        "tool_call_id": tc.id
                    })
                elif tc.function.name == "dice_roller":
                    tool_result = dice_roll(json.loads(tc.function.arguments)["dice"])
                    conversations.append({
                        "role": "tool",
                        "content": str(tool_result),
                        "tool_call_id": tc.id
                    })
                total_tool_calls += 1
            if total_tool_calls >= 5:
                break
        # Else accept user input and continue the loop
        else:
            user_input = input(response.choices[0].message.content)
            if user_input == "stop" or user_input == "end of turn":
                return user_input, conversations
            conversations.append({
                "role": "user",
                "content": user_input
        })

In [49]:
max_iter = 10
conversations = []
while True:
    max_iter -= 1
    # Player turn
    combat_starter_message = [
        {
            "role": "developer",
            "content": f"""
            {player_message_content}

            Players:
            {pc_info}
            """       
        },
    ]
    user_input = input("What would you like to do?")
    if user_input == "stop":
        break
    if user_input != "end of turn":
        conversations.append({"role": "user", "content": user_input})
        stop_code, conversations = resolve_attack(combat_starter_message, pc_tools, conversations)
    
    # Enemy turn 
    if stop_code == "stop":
        break
    enemy_starter_message = [
        {
            "role": "developer",
            "content": f"""
            {npc_message_content}

            NPC:
            {npc_info}
            """
        }
    ]
    print(f"The total conversation so far is: {conversations}")
    stop_code, conversations = resolve_attack(enemy_starter_message, npc_tools, conversations)
    if stop_code == "stop" or max_iter == 0:
        break    

Content of message: {'role': 'user', 'content': 'I will attack the Imp with my shortsword'}
Response is: ChatCompletion(id='chatcmpl-AjrIo4DVgaDWBT2QZoGGUcZrqKB74', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='You draw your shortsword and charge towards the Imp, aiming to strike. Roll to hit by making an attack roll, which consists of a d20 plus your attack modifier. Since you are proficient with shortswords, your attack modifier is your Dexterity modifier (+3) plus your proficiency bonus (+2). \n\nPlease roll a d20 and add 5 to the result.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1735493018, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_d02d531b47', usage=CompletionUsage(completion_tokens=77, prompt_tokens=399, total_tokens=476, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, r

In [54]:
max_iter = 15
conversations = []
while True:
    max_iter -= 1
    # Player turn
    combat_starter_message = [
        {
            "role": "developer",
            "content": f"""
            You are a Dungeon Master overseeing a combat encounter. The player and NPC will take turns making decisions and resolving actions.
            When it is the player's turn, you will ask for their action and you will determine the outcome.
            When it is the NPC's turn, you will determine the NPC's action and the outcome.

            If the player needs to make a roll (even if it is as a result of an NPC's action), you will ask the player for the roll.
            If the NPC needs to make a roll (even if it is as a result of a player's action), use the dice_roller tool.
            
            IMPORTANT: Use only the information you have been given to narrate the outcome. Whenever a question cannot be resolved with the information
            you have been given, you MUST use the consult_players_handbook tool to get up-to-date information. This includes any information about the rules of the game,
            the setting, the mechanics, NPC stat sheets, and so on.

            Players:
            {pc_info}

            NPCs:
            {npc_info}
            """       
        },
    ]
    user_input = input("What would you like to do?")
    if user_input == "stop":
        break
    if user_input != "end of turn":
        conversations.append({"role": "user", "content": user_input})
        conversations.append({"role": "developer", "content": consult_players_handbook("What rules do I need to know to resolve a turn in combat?")})
        stop_code, conversations = resolve_attack(combat_starter_message, npc_tools, conversations)
    
    print(f"The total conversation so far is: {conversations}")
    if stop_code == "stop" or max_iter == 0:
        break  

/home/vscode/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.153'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Content of message: {'role': 'developer', 'content': "To resolve a turn in combat, you need to follow these steps:\n\n1. **Movement**: You can move a distance up to your speed. Decide whether to move first or take your action first.\n\n2. **Actions**: You can take one action on your turn. Common actions include:\n   - **Attack**: Make one melee or ranged attack.\n   - **Cast a Spell**: Use your action to cast a spell with the appropriate casting time.\n   - **Dash, Disengage, Dodge, Help, Hide, Ready, Search, Use an Object**: Choose any of these for your action.\n\n3. **Bonus Actions**: If you have any abilities or spells that allow for a bonus action, you can take one in addition to your regular action.\n\n4. **Other Activities**: You can communicate freely, interact with one object as part of your move or action, or take a reaction in response to specific triggers (but only one reaction until your next turn).\n\n5. **Resolve Actions**: If you make an attack, roll a d20, add any relev